In [8]:
import openmm as mm
from openmm import app, unit
from openmm.app import PDBFile
from openmm.app import ForceField, Topology

In [9]:
with open('system_with_ligand_only.xml', 'r') as f:
    system_1 = mm.XmlSerializer.deserialize(f.read())
pdb = PDBFile('ligand_in_solvent.pdb')
system_1.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 300*unit.kelvin))

5

In [12]:
#run this cell under grappa conda environment
import grappa
from grappa import OpenmmGrappa
# Load the pretrained ML model from a tag.
grappa_ff = OpenmmGrappa.from_tag('grappa-1.4.1-radical')

# Overwrite bonded parameters with those predicted by grappa.
# Grappa will neither change the ion or solvent nor the nonbonded parameters.
system_1 = grappa_ff.parametrize_system(system_1, pdb.topology, plot_dir='.')

INFO:root:Initializing model with tag grappa-1.4.1-radical...
INFO:root:Downloading model grappa-1.4.1-radical from https://github.com/graeter-group/grappa/releases/download/v.1.4.1/grappa-1.4.1-radical.zip and unzipping it to:
	/home/haolan/.grappa_cache/models/grappa-1.4.1-radical
100%|██████████| 396M/396M [00:25<00:00, 15.3MB/s] 
INFO:root:Downloaded to /home/haolan/.grappa_cache/models/grappa-1.4.1-radical
INFO:root:Loading model from /home/haolan/.grappa_cache/models/grappa-1.4.1-radical/epoch:179-early_stop_loss:18.59.ckpt


NameError: name 'get_subtopology' is not defined

In [ ]:
#NPT simulations for 10 ns
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(pdb.topology, system_1, integrator)
#
simulation.context.setPositions(pdb.positions)
integrator.setRandomNumberSeed(42)
print("Simulation is running on:", simulation.context.getPlatform().getName())
simulation.minimizeEnergy(maxIterations=1000)
# save the minimized structure for each replicate as starting-point geometry (almost identical actually...)
with open(f'min_complex.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), f)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
# NPT for 10 ns
simulation.reporters.append(app.DCDReporter(f'trajectory.dcd', 5000))
simulation.reporters.append(
                            app.StateDataReporter(
                                                    f'energy_data.csv',
                                                    5000,
                                                    step=True,
                                                    time=True, 
                                                    potentialEnergy=True,
                                                    kineticEnergy=True,
                                                    totalEnergy=True,
                                                    temperature=True,
                                                    volume=True,
                                                    density=True,
                                                    speed=True,
                                                    )
                            )
simulation.step(5000000)
simulation.reporters.clear()

Simulation is running on: CUDA
